# Session 3 - P4D Simulations

Let's import BattMo and some other packages into the kernel.

In [1]:
using BattMo, GLMakie

GLMakie.activate!(inline=false)

## Pouch cell

- C : electrolyte concentration
- Cp: particle concentration
- Cs: surface concentration
- Phi: potential

In [ ]:
cell_parameters = load_cell_parameters(; from_default_set = "Xu2015")


KeyError: KeyError: key "ElectrolyteConductivity" not found

In [ ]:

cycling_protocol = load_cycling_protocol(; from_default_set = "CCDischarge")
model_settings = load_model_settings(; from_default_set = "P4D_pouch")
simulation_settings = load_simulation_settings(; from_default_set = "P4D_pouch")

model_setup = LithiumIonBattery(; model_settings)

sim = Simulation(model_setup, cell_parameters, cycling_protocol; simulation_settings);
output = solve(sim)

plot_interactive_3d(output; colormap = :curl)

✔️ Validation of ModelSettings passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CellParameters passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CyclingProtocol passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of SimulationSettings passed: No issues found.
──────────────────────────────────────────────────
Jutul: Simulating 2 hours, 12 minutes as 163 report steps


Progress   1%|█                                          |  ETA: 0:01:56

Progress   2%|█                                          |  ETA: 0:02:40

Progress   2%|██                                         |  ETA: 0:03:20

Progress   3%|██                                         |  ETA: 0:03:12

Progress   4%|██                                         |  ETA: 0:02:54

Progress   4%|██                                         |  ETA: 0:02:53

Progress   5%|███                                        |  ETA: 0:02:50

Progress   5%|███                                        |  ETA: 0:02:38

Progress   6%|███                                        |  ETA: 0:02:38

Progress   7%|███                                        |  ETA: 0:02:35

Progress   7%|████                                       |  ETA: 0:02:31

Progress   8%|████                                       |  ETA: 0:02:27

Progress   9%|████                                       |  ETA: 0:02:27

Progress   9%|████                    

╭────────────────┬───────────┬───────────────┬──────────╮
│ Iteration type │  Avg/step │  Avg/ministep │    Total │
│                │ 138 steps │ 138 ministeps │ (wasted) │
├────────────────┼───────────┼───────────────┼──────────┤
│ Newton         │   2.13043 │       2.13043 │  294 (0) │
│ Linearization  │   3.13043 │       3.13043 │  432 (0) │
│ Linear solver  │   2.13043 │       2.13043 │  294 (0) │
│ Precond apply  │       0.0 │           0.0 │    0 (0) │
╰────────────────┴───────────┴───────────────┴──────────╯
╭───────────────┬──────────┬────────────┬──────────╮
│ Timing type   │     Each │   Relative │    Total │
│               │       ms │ Percentage │        s │
├───────────────┼──────────┼────────────┼──────────┤
│ Properties    │   6.8012 │     1.20 % │   1.9996 │
│ Equations     │  11.4568 │     2.97 % │   4.9493 │
│ Assembly      │   1.0143 │     0.26 % │   0.4382 │
│ Linear solve  │ 539.8135 │    95.28 % │ 158.7052 │
│ Linear setup  │   0.0000 │     0.00 % │   0.0000 │
│

## Cylindrical cell

In [ ]:
cell_parameters = load_cell_parameters(; from_default_set = "Chen2020")
cycling_protocol = load_cycling_protocol(; from_default_set = "CCDischarge")
model_settings = load_model_settings(; from_default_set = "P4D_cylindrical")
simulation_settings = load_simulation_settings(; from_default_set = "P4D_cylindrical")

model_setup = LithiumIonBattery(; model_settings)

sim = Simulation(model_setup, cell_parameters, cycling_protocol; simulation_settings);
output = solve(sim)